# Vorbereitungen

In [ ]:
!apt-get install poppler-utils libpoppler-cpp-dev tesseract-ocr
!pip install -v -v python-poppler

In [ ]:
!pip install -U langchain langchain-community langchainhub langchain-openai chromadb==0.3.29 wikipedia-api gradio==3.48.0 pydantic sqlalchemy==2.0.0 unstructured[pdf]

In [ ]:
# OPENAI KEY lesen
import os
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get('OPENAI_KEY')
except:
    OPENAI_KEY = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


# Beispiel-PDF

In [ ]:
from pathlib import Path
import gradio as gr

In [ ]:
Path('data/pdf').mkdir(parents=True, exist_ok=True)

In [ ]:
# download Beispiel PDF
!wget https://github.com/floleuerer/elvtr/blob/main/session_06/data/pdf/19364-Article%20Text-23377-1-2-20220531.pdf?raw=True -O data/pdf/19364-Article%20Text-23377-1-2-20220531.pdf

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
loader = DirectoryLoader('data/pdf/')
docs = loader.load()
len(docs)

# LangChain Conversational RAG

In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

## Preprocessing - Dokument laden

Wir verwenden in dem Beispiel Wikipedia-Artikel - hier könnte aber auch der "PDF"-Teil von ganz unten im Notebook eingefügt werden um PDFs zu verwenden.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
loader = DirectoryLoader('data/pdf/')
docs = loader.load()
len(docs)

## Preprocessing - Dokument chunken

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc_chunks = text_splitter.split_documents(docs)

In [ ]:
doc_chunks[:4]

## Chunks embedden und in Vektor-Datenbank speichern

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [ ]:
# .chroma/ ordern löschen der evtl noch "alte" Dokumente enthält
!rm -rf .chroma/

In [ ]:
vectorstore = Chroma.from_documents(documents=doc_chunks, embedding=OpenAIEmbeddings(model='text-embedding-3-small'))
retriever = vectorstore.as_retriever()

## Q&A - Passende Antwort mit LLM erzeugen lassen

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage

In [ ]:
llm = ChatOpenAI(model='gpt-3.5-turbo')

### Retriever Chain

In [ ]:
retriever_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, retriever_prompt)

### Document + Retrieval Chains

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

# RAG mit Gradio-GUI

In [ ]:
def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    output = retrieval_chain.invoke({
        "chat_history": history_langchain_format,
        "input": message
    })
    return output['answer']

gr.ChatInterface(predict).launch()